In [200]:
# import numpy

#todo: how many attacks before shuffle will trigger
#will not do: AOEs can trrigger the shuffle but won' thappen until the end of the round. i don't care.
#add effects like wound and poison and all that. no change in expected dmg but it's worth reporting
#calculate avg damage based on different base damages?
#add blesses? how... +1 bless card added to deck per x attacks maybe?

from collections import defaultdict
from datetime import datetime
import random
from pprint import pprint

In [3]:
defaultdeck = ['null',-2,-1,-1,-1,-1,-1,0,0,0,0,0,0,1,1,1,1,1,2,'crit']

In [274]:
perklist = {'Brute':
            [['remove',[-1,-1]], #remove 2 -1s
              ['replace',[-1,-1],[1,1]], #replace -1 with +1
              ['add',[1,1]], #add 2+1
              ['add',[1,1]], #add 2+1s
              ['add',[3]], #add one #3
              ['add',['0-Push 1-R','0-Push 1-R','0-Push 1-R']], #add 3 rolling push1
              ['add',['0-Push 1-R','0-Push 1-R','0-Push 1-R']], #add 3 rolling push1
              ['add',['0-Pierce 2-R','0-Pierce 2-R']], #add 2 pierce3
              ['add',['0-Stun-R']], #add rolling stun
              ['add',['0-Stun-R']], #add rolling stun
              ['add',['0-Disarm-R','0-Muddle-R']], #add rolling disarm and rolling muddle
              ['add',['0-Add Target-R']], #add rolling add target
              ['add',['0-Add Target-R']], #add rolling add target
              ['add',['0-Shield 1']], # add 1 +1 shield self
              ['add',[1]] #ignore neg item effects and add +1 card
             ],
           'Spellweaver':
            [['remove',[0,0,0,0]], #remove 4 0s
             ['replace',[-1],[1]], #replace -1 with +1
             ['replace',[-1],[1]], #replace -1 with +1
             ['add',[1,1]], #add 2 +1s
             ['add',[1,1]], #add 2 +1s
             ['add',['0-Stun']], #add +0 stun
             ['add',['1-Wound']], #add +1 wound
             ['add',['1-Immobilize']], #add +1 immob
             ['add',['1-Curse']], #add +1 curse
             ['add',['2-Fire']], #add +2 fire
             ['add',['2-Fire']], #add +2 fire
             ['add',['2-Ice']], #add +2 frost
             ['add',['2-Ice']], #add +2 frost
             ['add',['0-Earth-R','0-Air-R']], #add rolling earth, rolling air
             ['add',['0-Light-R','0-Dark-R']] #add rolling light, rolling dark
            ],
           'Soothsinger':
            [['remove',[-1,-1]], #remove 2 -1
             ['remove',[-1,-1]], #remove 2 -1
             ['remove',[-2]], #remove 1 -2
             ['replace',[1,1],[4]], #replace 2 +1 with 1 +4
             ['replace',[1,1],[4]], #replace 2 +1 with 1 +4
             ['replace',[0],['1-Immobilize']],#replace 0 with 1 immob
             ['replace',[0],['1-Disarm']], #replace 0 with 1 disarm
             ['replace',[0],['2-Wound']], #replace 0 with 2 wound
             ['replace',[0],['2-Poison']], #replace 0 with 2 poison
             ['replace',[0],['2-Curse']], #replace 0 with 2 curse
             ['replace',[0],['3-Muddle']], #replace 0 with 3 muddle
             ['replace',[-1],['0-Stun']], #replace -1 with 0 stun
             ['add',['1--R','1--R','1--R']], #add 3 rolling +1
             ['add',['0-Curse-R','0-Curse-R']], #add 2 rolling cvurse
             ['add',['0-Curse-R','0-Curse-R']] #add 2 rolling curse
            ],
           'Elementalist':
            [['remove',[-1,-1]], #remove 2 -1
             ['remove',[-1,-1]], #remove 2 -1
             ['replace',[-1],[1]], #replace -1, +1
             ['replace',[0],[2]], #replace 0 with 2
             ['replace',[0],[2]], #replace 0 with +2
             ['add',['0-Fire','0-Fire','0-Fire']], #add 3 +0 fire
             ['add',['0-Ice','0-Ice','0-Ice']], #add 3 +0 frost
             ['add',['0-Air','0-Air','0-Air']], #add 3 +0 air
             ['add',['0-Earth','0-Earth','0-Earth']], #add 3 +0 earth
             ['replace',[0,0],['0-Fire','0-Earth']], #replace 2 +0 with +0 fire, +0 earth
             ['replace',[0,0],['0-Ice','0-Air']], #replace 2 +0 with +0 frost, +0 air
             ['add',['1-Push 1','1-Push 1']], #add two +1 push
             ['add',['1-Wound']], #add one +1 wound
             ['add',['0-Stun']], #add 1 +0 stun
             ['add',['0-Add Target']] #add 1 +0 additional target
            ],
            'Doomstalker':
            [['remove',[-1,-1]], #remove 2 -1
             ['remove',[-1,-1]], #remove 2 -1
             ['replace',[0,0],[1,1]], #replace 2 0 with 2 +1
             ['replace',[0,0],[1,1]], #replace 2 0 with 2 +1
             ['replace',[0,0],[1,1]],#replace 2 0 with 2 +1
             ['add',['1--R','1--R']], #add 2 rolling +1
             ['add',['1--R','1--R']],#add 2 rolling +1
             ['add',['2-Muddle']], #add +2 muddle
             ['add',['1-Poison']], #add +1 poison
             ['add',['1-Wound']], #add +1 wound
             ['add',['1-Immobilize']], #add +1 immob
             ['add',['0-Stun']], #add +0 stun
             ['add',['0-Add Target-R']],  #add 1 rolling add target
             ['add',['0-Add Target-R']],  #add 1 rolling add target
             ['add',[]] #ignore neg scena
            ],
            'Sawbones':
            [['remove',[-1,-1]], #remove2 -1
             ['remove',[-1,-1]], #remove2 -1
             ['remove',[0,0,0,0]], #remove 4 0
             ['replace',[0],[2]], #replace 0 with +2
             ['replace',[0],[2]], #replace 0 with +2
             ['add',['2--R']],#add 1 +2R
             ['add',['2--R']],#add 1 +2R
             ['add',['1-Immobilize']],#add 1 +1 immo
             ['add',['1-Immobilize']],#add 1 +1 immo
             ['add',['0-Wound-R','0-Wound-R']], #add 2 rolling wound
             ['add',['0-Wound-R','0-Wound-R']], #add 2 rolling wound
             ['add',['0-Stun-R']], #add one rolling stun
             ['add',['0-Heal 3-R']],#add rolling heal 3
             ['add',['0-Heal 3-R']],#add rolling heal 3
             ['add',['0-Refresh Item']], #add one +0 refresh an item
            ],
            'BeastTyrant':
            [['remove',[-1,-1]], #remove2 -1
             ['replace',[-1],[1]], #replace -1 +1
             ['replace',[-1],[1]], #replace -1 +1
             ['replace',[-1],[1]], #replace -1 +1
             ['replace',[0],[2]], #replace 0 with +2
             ['replace',[0],[2]], #replace 0 with +2
             ['add',['1-Wound']],#add 1 +1 wound
             ['add',['1-Wound']],#add 1 +1 wound
             ['add',['1-Immobilize']],#add 1 +1 immo
             ['add',['1-Immobilize']],#add 1 +1 immo
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal+1s
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal+1s
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal+1s
             ['add',['0-Earth-R','0-Earth-R']], #add 2 rolling earth
             ['add',[]], #ignore neg scenario effects
            ],
            'Berserker':
            [['remove',[-1,-1]], #remove2 -1
             ['remove',[0,0,0,0]], #remove 4 0s
             ['replace',[-1],[1]], #replace -1 +1
             ['replace',[-1],[1]], #replace -1 +1
             ['replace',[0],['2--R']], #replace 0 with +2Rolling
             ['replace',[0],['2--R']], #replace 0 with +2rolling
             ['add',['0-Wound-R','0-Wound-R']],#add 2 rolling wound
             ['add',['0-Wound-R','0-Wound-R']],#add 2 rolling wound
             ['add',['0-Stun-R']],#add 1 rolling stun
             ['add',['0-Stun-R']],#add 1 rolling stun
             ['add',['1-Disarm-R']], #add 1 rolling +1 disarm
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal +1
             ['add',['2-Fire','2-Fire']], #add 2 +2 fires
             ['add',['2-Fire','2-Fire']], #add 2 +2 fires
             ['add',[]], #ignore neg scenario effects
            ],
            'Plagueherald':
            [['replace',[-2],[0]], #replace -2 with +0
             ['replace',[-1],[1]], #replace -1, +1
             ['replace',[-1],[1]], #replace -1, +1
             ['replace',[0],[2]], #replace 0, 2
             ['replace',[0],[2]], #replace 0, 2
             ['add',[1,1]], #add 2 +1
             ['add',['1-Air']],#add +1 air
             ['add',['1-Air']],#add +1 air
             ['add',['1-Air']],#add +1 air
             ['add',['0-Poison-R','0-Poison-R','0-Poison-R']],#add 3 rolling poisons
             ['add',['0-Curse-R','0-Curse-R']], #add 2 rolling curses
             ['add',['0-Immobilize-R','0-Immobilize-R']], #add 2 rolling immobilizes
             ['add',['0-Stun-R']], #add one rolling stun
             ['add',['0-Stun-R']], #add one rolling stun
             ['add',[1]], #ignore neg scenario effects and add one +1
            ],
            'Nightshroud':
            [['remove',[-1,-1]], #remove2 -1
             ['remove',[-1,-1]], #remove2 -1
             ['remove',[0,0,0,0]], #remove4 0
             ['add',['-1-Dark']], #add 1 -1 DARK
             ['add',['-1-Dark']], #add 1 -1 DARK
             ['replace',['-1-Dark'],['1-Dark']], #replace -1 DARK with +1 DARK
             ['replace',['-1-Dark'],['1-Dark']], #replace -1 DARK with +1 DARK
             ['add',['1-Invisibility']],#add +1 INVIS
             ['add',['1-Invisibility']],#add +1 INVIS
             ['add',['0-Muddle-R','0-Muddle-R','0-Muddle-R']],#add 3 rolling muddles
             ['add',['0-Muddle-R','0-Muddle-R','0-Muddle-R']],#add 3 rolling muddles
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal 1
             ['add',['0-Curse-R','0-Curse-R']], #add 2 rolling curse
             ['add',['0-Add Target-R']], #add rolling add target
             ['add',[1,1]], #ignore neg scenario effects and add two +1
            ],
            'Summoner':
            [['remove',[-1,-1]], #remove2 -1
             ['replace',[-2],[0]], #replace -2 with +0
             ['replace',[-1],[1]], #replace -1 with +1
             ['replace',[-1],[1]], #replace -1 with +1
             ['replace',[-1],[1]], #replace -1 with +1
             ['add',[2]], #add +2
             ['add',[2]], #add +2
             ['add',['0-Wound-R','0-Wound-R']],#add 2 rolling wound
             ['add',['0-Poison-R','0-Poison-R']],#add 2 rolling poison
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal 1
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal 1
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal 1
             ['add',['0-Fire-R','0-Air-R']], #add 1 rolling fire, rolling air
             ['add',['0-Dark-R','0-Earth-R']], #add 1 rolling night, rolling earth
             ['add',[1,1]], #ignore neg scenario effects and add two +1
            ],
            'Quartermaster':
            [['remove',[-1,-1]], #remove2 -1
             ['remove',[-1,-1]], #remove2 -1
             ['remove',[0,0,0,0]], #remove4 0
             ['replace',[0],[2]], #replace 0 with 2
             ['replace',[0],[2]], #replace 0 with 2
             ['add',['1--R','1--R']],#add 2 rolling +1s
             ['add',['1--R','1--R']],#add 2 rolling +1s
             ['add',['0-Muddle-R','0-Muddle-R','0-Muddle-R']],#add 3 rolling muddles
             ['add',['0-Pierce 3-R','0-Pierce 3-R']], #add 2 rolling pierce 3
             ['add',['0-Stun-R']], #add 1 rolling stun
             ['add',['0-Add Target-R']], #add 1 rolling add target
             ['add',['0-Refresh an item']], #add 1 +0  refresh an item
             ['add',['0-Refresh an item']], #add 1 +0  refresh an item
             ['add',['0-Refresh an item']], #add 1 +0  refresh an item
             ['add',[1,1]], #ignore neg item effects and add two +1
            ],
            'Sunkeeper':
            [['remove',[-1,-1]], #remove2 -1
             ['remove',[-1,-1]], #remove2 -1
             ['remove',[0,0,0,0]], #remove4 0
             ['replace',[-2],[0]], #replace 0 with 2
             ['replace',[0],[2]], #replace 0 with 2
             ['add',['1--R','1--R']],#add 2 rolling +1s
             ['add',['1--R','1--R']],#add 2 rolling +1s
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal+1
             ['add',['0-Heal 1-R','0-Heal 1-R']], #add 2 rolling heal+1
             ['add',['0-Stun-R']], #add 1 rolling stun
             ['add',['0-Light-R','0-Light-R']], #add 2 rolling light
             ['add',['0-Light-R','0-Light-R']], #add 2 rolling light
             ['add',['0-Shield 1-R','0-Shield 1-R']], #add 2 sheidl self
             ['add',[1,1]], #ignore neg scenario effects and add two +1
             ['add',[]], #ignore scenario effects
            ],
            'Mindthief':
            [['remove',[-1,-1]], #remove2 -1
             ['remove',[-1,-1]], #remove2 -1
             ['remove',[0,0,0,0]], #remove4 0
             ['replace',[1,1],[2,2]], #replace 2 1s, with 2 2s
             ['replace',[-2],[0]], #replace -2, 0
             ['add',['2-Ice']],#add +2 frost
             ['add',['2-Ice']],#add +2 frost
             ['add',['1--R','1--R']],#add 2 rolling +1s
             ['add',['1--R','1--R']],#add 2 rolling +1s
             ['add',['0-Push 1-R','0-Push 1-R','0-Push1-R']], #add 3 rolling push1
             ['add',['0-Muddle-R','0-Muddle-R','0-Muddle-R']], #add 3 rolling muddles
             ['add',['0-Immobilize-R','0-Immobilize-R']], #add 2 rolling immobilize
             ['add',['0-Stun-R']], #add 1 rolling stun
             ['add',['0-Disarm-R','0-Muddle-R']], #add 1 rolling =disarm and 1 rolling muddle
             ['add',[]], #ignore scenario effects
            ],
            'Mindthief':
            [['remove',[-1,-1]], #remove2 -1
             ['remove',[-1,-1]], #remove2 -1
             ['remove',[0,0,0,0]], #remove4 0
             ['replace',[1,1],[2,2]], #replace 2 1s, with 2 2s
             ['replace',[-2],[0]], #replace -2, 0
             ['add',['2-Ice']],#add +2 frost
             ['add',['2-Ice']],#add +2 frost
             ['add',['1--R','1--R']],#add 2 rolling +1s
             ['add',['1--R','1--R']],#add 2 rolling +1s
             ['add',['0-Push 1-R','0-Push 1-R','0-Push 1-R']], #add 3 rolling push1
             ['add',['0-Muddle-R','0-Muddle-R','0-Muddle-R']], #add 3 rolling muddles
             ['add',['0-Immobilize-R','0-Immobilize-R']], #add 2 rolling immobilize
             ['add',['0-Stun-R']], #add 1 rolling stun
             ['add',['0-Disarm-R','0-Muddle-R']], #add 1 rolling =disarm and 1 rolling muddle
             ['add',[]], #ignore scenario effects
            ],
            'Cragheart':
            [['remove',[0,0,0,0]], #remove4 0
             ['replace',[-1],[1]], #replace -1, +1
             ['replace',[-1],[1]], #replace -1, +1
             ['replace',[-1],[1]], #replace -1, +1
             ['add',[-2,2,2]],#add -2,+2,+2
             ['add',['1-Immobilize']],#add +1 immob
             ['add',['1-Immobilize']],#add +1 immob
             ['add',['2-Muddle']],#add +2 muddle
             ['add',['2-Muddle']],#add +2 muddle
             ['add',['0-Push 2-R','0-Push 2-R']], #add 2 rolling push2
             ['add',['0-Earth-R','0-Earth-R']], #add 2 rolling earth
             ['add',['0-Earth-R','0-Earth-R']], #add 2 rolling earth
             ['add',['0-Air-R','0-Air-R']], #add 2 rolling air
             ['add',[]], #ignore item effects
             ['add',[]], #ignore scenario effects
            ],
            'Scoundrel':
            [['remove',[-1,-1]], #remove 2 -1s
             ['remove',[-1,-1]], #remove 2 -1s
             ['remove',[0,0,0,0]], #remove 4 0s
             ['replace',[-2],[0]], #replace -2 with 0
             ['replace',[-1],[1]], #replace -1, +1
             ['replace',[0],[2]], #replace 0 with 2
             ['replace',[0],[2]], #replace 0 with 2
             ['add',['1--R','1--R']], #add 2 rolling +1
             ['add',['1--R','1--R']], #add 2 rolling +1
             ['add',['0-Pierce 3-R','0-Pierce 3-R']], #add 2 rolling pierce3
             ['add',['0-Poison-R','0-Poison-R']], #add 2 rolling poison
             ['add',['0-Poison-R','0-Poison-R']], #add 2 rolling poison
             ['add',['0-Muddle-R','0-Muddle-R']], #add 2 rolling muddle
             ['add',['0-Invisibility-R']], #add 1 rolling invis
             ['add',[]], #ignore scenario effects
            ],
            'Tinkerer':
            [['remove',[-1,-1]], #remove 2 -1s
             ['remove',[-1,-1]], #remove 2 -1s
             ['replace',[-2],[0]], #replace -2 with 0
             ['add',[1,1]], #add 2+1s
             ['add',[3]], #add 1 +3
             ['add',['0-Fire-R','0-Fire-R']], #add 2 rolling FIRE
             ['add',['0-Muddle-R','0-Muddle-R','0-Muddle-R']], #add 2 rolling muddle
             ['add',['1-Wound']], #add 1 +1 wound
             ['add',['1-Wound']], #add 1 +1 wound
             ['add',['1-Immobilize']], #add 1 +1 immo
             ['add',['1-Immobilize']], #add 1 +1 immo
             ['add',['1-Heal 2']], #add 1 +1 heal+2
             ['add',['1-Heal 2']], #add 1 +1 heal+2
             ['add',['0-Add Target']], #add 1 +0 add target
             ['add',[]], #ignore scenario effects
            ],
           }
class_list = ['Brute','Spellweaver','Soothsinger','Elementalist','Doomstalker', 'Sawbones','BeastTyrant','Berserker',
             'Plagueherald','Nightshroud','Summoner','Quartermaster','Sunkeeper','Mindthief', 'Cragheart', 'Scoundrel',
             'Tinkerer']

In [280]:
%%capture cap --no-stderr
num_attacks = 100000
defattack = 2
advantage = 0
disadvantage = 0
print(f'Number of attacks: {num_attacks}')
print(f'Base attack damage: {defattack}')
print(f'Percentage of attacks at advantage: {advantage*100}%')
print(f'Percentage of attacks at disadvantage: {disadvantage*100}%')
print('-'*25)
start_time = datetime.now()
# class_list = ['Elementalist']
for eachclass in class_list:
    totaldmg = 0
    isadv = 0
    isdisadv = 0
    perks = perklist[eachclass]

    defaultdeck = ['null',-2,-1,-1,-1,-1,-1,0,0,0,0,0,0,1,1,1,1,1,2,'crit']
    modded_deck = defaultdeck.copy()
    perks_chosen = []

    for i in perks_chosen:
        if perks[i-1][0] == 'remove':
            for j in perks[i-1][1]:
                modded_deck.remove(j)
        elif perks[i-1][0] == 'add':
            for j in perks[i-1][1]:
                modded_deck.append(j)
        elif perks[i-1][0] == 'replace':
            for j in perks[i-1][1]:
                modded_deck.remove(j)
            for j in perks[i-1][2]:
                modded_deck.append(j)
    # print(modded_deck)

    numcrits = 0
    numnulls = 0
    shuffle = True
    dict_shuffles = defaultdict(int)
    modifiers = defaultdict(int)
    num_turns_to_shuffle = 0
    for i in range(num_attacks):
        crit = False
        null = False
        rolling_mod = False
        if shuffle:
            moddeck = modded_deck.copy()
            random.shuffle(moddeck)
            dict_shuffles[num_turns_to_shuffle] += 1
            num_turns_to_shuffle = 0
            shuffle = False
#         print(moddeck)
        draw = []
        isadvdisadv = random.random()
        added_dmg = defattack
        if isadvdisadv <= advantage:
            advdisadv = 'A'
            isadv += 1
        elif isadvdisadv <= advantage + disadvantage:
            advdisadv = 'D'
            isdisadv += 1
        else:
            advdisadv = 'N'
#         print(advdisadv)
        draw.append(moddeck.pop())
        
        while str(draw[-1])[-1] == 'R':
            rolling_mod = True
            draw.append(moddeck.pop())
        
        if (advdisadv == 'A' or advdisadv == 'D') and len(draw) == 1:
            draw.append(moddeck.pop())
            if str(draw[-1])[-1] == 'R':
                rolling_mod = True

        draw.sort(key = lambda item: ([int,str].index(type(item)), item))
#         print(draw)       
        if advdisadv == 'N':
            if rolling_mod == True:
                if 'crit' in draw:
                    shuffle = True
                    draw.pop()
                    for each_card in draw:
                        added_dmg += int(each_card[0])
                        if '-' in each_card:
                            modifiers[each_card.split('-')[1]] += 1
                    crit = True

                elif 'null' in draw:
                    shuffle = True
                    draw.pop()
                    for each_card in draw:
                        if '-' in each_card:
                            modifiers[each_card.split('-')[1]] += 1
                    null = True
                else:
                    for each_card in draw:
                        if type(each_card) == int:
                            added_dmg += each_card
                        else:
                            added_dmg += int(each_card.split('-')[0])
                            modifiers[each_card.split('-')[1]] += 1
#                             print('each_card.split('-')[1]')
            else:
                if draw[0] == 'crit':
                    crit = True
                    shuffle = True
                elif draw[0] == 'null':
                    null = True
                    shuffle = True
                else:
                    if type(draw[0]) == int:
                        added_dmg += draw[0]
                    else:
                        added_dmg += int(draw[0].split('-')[0])
                        modifiers[draw[0].split('-')[1]] += 1
                        
        elif advdisadv == 'D':
            if 'null' in draw:
                null = True
                shuffle = True
            else:
                if rolling_mod == True:
                    if 'crit' in draw:     #crit on disadvantage, ignore all rolling
                        crit = True
                        shuffle = True
                    else:       #non crit at disadvantage, ignore all rolling, just find the one non-rolling card
                        for each_card in draw:
                            if type(each_card) == int:
                                added_dmg += each_card
                                break
                            elif each_card[-1] != 'R':
                                added_dmg += int(each_card.split('-')[0])
                                modifiers[each_card.split('-')[1]] += 1
                                break
                else:
                    if 'crit' in draw:
                        shuffle = True
                        atk_val = draw[0] if type(draw[0])==int else int(draw[0].split('-')[0])
                        
                        if added_dmg <= atk_val:
                            crit = True
                        else:
                            added_dmg += atk_val
                    else:
                        card1 = [draw[0],None] if type(draw[0])==int else [int(draw[0].split('-')[0]),draw[0].split('-')[1]]
                        card2 = [draw[1],None] if type(draw[1])==int else [int(draw[1].split('-')[0]),draw[1].split('-')[1]]
                        if card1[0] < card2[0]:
                            added_dmg += card1[0]
                            if card1[1] != None:
                                modifiers[card1[1]] += 1
                        elif card2[0] < card1[0]:
                            added_dmg += card2[0]
                            if card2[1] != None:
                                modifiers[card2[1]] += 1
                        else:
                            if card1[1] == None:
                                added_dmg += card1[0]
                            elif card2[1] == None:
                                added_dmg += card2[0]
                            else:
                                added_dmg += card1[0]
                                modifiers[card1[1]] += 1
                                
                        
        elif advdisadv == 'A':
            if rolling_mod == True:
                if 'crit' in draw:
                    shuffle = True
                    crit = True
                    draw.pop()
                    for each_card in draw:
                        added_dmg += int(each_card.split('-')[0])
                        modifiers[each_card.split('-')[1]] += 1       
                elif 'null' in draw:
                    null = True
                    shuffle = True
                    draw.remove('null')
                    for each_card in draw:
                        if type(each_card) == str:
                            modifiers[each_card.split('-')[1]] += 1                
                else:
                    for each_card in draw:
                        if type(each_card) == int:
                            added_dmg += each_card
                        else:
                            added_dmg += int(each_card.split('-')[0])
                            modifiers[each_card.split('-')[1]] += 1
                        
            else:
                if 'crit' in draw and 'null' in draw:
                    shuffle = True
                    crit = True
                elif 'crit' in draw: #evaluate which is bigger
                    shuffle = True
                    draw.remove('crit')
                    if added_dmg >= (draw[0] if type(draw[0])==int else int(draw[0].split('-')[0])):
                        crit = True
                    else:
                        added_dmg += draw[0] if type(draw[0])==int else int(draw[0].split('-')[0])
                    
                elif 'null' in draw:
                    shuffle = True
                    draw.remove('null')
                    if type(draw[0]) == int:
                        added_dmg += draw[0]
                    else:
                        added_dmg += int(draw[0].split('-')[0])
                        modifiers[draw[0].split('-')[1]] += 1
                else:
                    card1 = [draw[0],None] if type(draw[0])==int else [int(draw[0].split('-')[0]),draw[0].split('-')[1]]
                    card2 = [draw[1],None] if type(draw[1])==int else [int(draw[1].split('-')[0]),draw[1].split('-')[1]]
                    if card1[0] > card2[0]:
                        added_dmg += card1[0]
                        if card1[1] != None:
                            modifiers[card1[1]] += 1
                    elif card2[0] > card1[0]:
                        added_dmg += card2[0]
                        if card2[1] != None:
                            modifiers[card2[1]] += 1
                    else:
                        if card1[1] != None:
                            added_dmg += card1[0]
                            modifiers[card1[1]] += 1
                        elif card2[1] != None:
                            added_dmg += card2[0]
                            modifiers[card2[1]] += 1
                        else:
                            added_dmg += card1[0]

                    
        if crit:
            added_dmg = added_dmg * 2
            numcrits += 1
        if null:
            added_dmg = 0
            numnulls += 1
        if added_dmg < 0:
            added_dmg = 0
#         print(added_dmg)
        totaldmg += added_dmg
        num_turns_to_shuffle += 1
#         print(num_turns_to_shuffle)
    if '' in modifiers.keys():
        del modifiers['']
#         print('-'*25)
    del dict_shuffles[0]
    print(f'Class: {eachclass}')
    print(f'Average total damage: {totaldmg/(num_attacks)}')
    print(f'Number of crits: {numcrits} ({"{:.2f}".format(numcrits/(num_attacks)*100)}%)')
    print(f'Number of nulls: {numnulls} ({"{:.2f}".format(numnulls/(num_attacks)*100)}%)')
    for key,val in sorted(dict_shuffles.items()):
        print(f'{key} attacks to trigger a shuffle: {"{:.2f}".format(val/sum(dict_shuffles.values())*100)}%')
    for key,val in modifiers.items():
        print(f'{key} on {"{:.2f}".format(val/num_attacks*100)}%')
    print('-'*25)
    
    
print(f'Time to run script: {datetime.now()-start_time}')

In [281]:
with open ('output.txt','w') as f:
    f.write(cap.stdout)

In [236]:
moddeck

[]

In [74]:
testdict = defaultdict(int)
testdict['asdfasdf'] += 1

In [252]:
dict_shuffles

defaultdict(int,
            {9: 584,
             20: 190,
             18: 281,
             13: 440,
             6: 667,
             5: 757,
             10: 520,
             12: 464,
             14: 425,
             7: 727,
             8: 613,
             3: 770,
             2: 813,
             1: 830,
             15: 397,
             16: 306,
             4: 726,
             17: 299,
             11: 457,
             19: 273,
             22: 148,
             21: 186,
             25: 36,
             24: 71,
             23: 90})

In [ ]:
default

In [214]:
dictionarytest = {'': 1568,
             'Curse': 2833,
             'Disarm': 766,
             'Immobilize': 691,
             'Muddle': 738,
             'Poison': 731,
             'Stun': 758,
             'Wound': 689}
del dictionarytest['']
dictionarytest

{'Curse': 2833,
 'Disarm': 766,
 'Immobilize': 691,
 'Muddle': 738,
 'Poison': 731,
 'Stun': 758,
 'Wound': 689}

In [145]:
a = '0-Shield 1'
b = 2

In [153]:
if (a if type(a)==int else int(a.split('-')[0])) < b:
    print(1)

In [169]:
if 1==1 and not 1==2:
    print(1)

1


In [22]:
# test = [1,2,3,4,5,6,7,8,9,10]
test.pop()
test

IndexError: pop from empty list

In [122]:
test = -1
test.split('-')

AttributeError: 'int' object has no attribute 'split'

In [279]:
resolve = ['crit','0R','1R','1R','null',0,1,2,3]
resolve.sort(key = lambda item: ([str,int].index(type(item)), item))
resolve

['0R', '1R', '1R', 'crit', 'null', 0, 1, 2, 3]

In [120]:
c = ['c', 'b', 'd', 'a', 4, 2, 1, 3]
c[-2]

1

In [285]:
moddeck[-1]

'0R'

In [287]:
len([1])

1

In [47]:
draw = [1, 0, 'null', 1, 2, '0-Push1-R', 0, 1, '0-Pierce2-R', '0-Stun-R', 1, 1, 0, '0-Push1-R', 0, 1, '0-Push1-R', 1, '0-Push1-R', 0, '0-Push1-R', 'crit', -1, '0-Disarm-R', 1, '0-Push1-R', 0, '0-Stun-R', -2, 1, 1, '0-Shield', 1, '0-AddTarget-R', '0-AddTarget-R', '0-Pierce2-R', 1, '0-Muddle-R']
draw.sort(key = lambda item: ([int,str].index(type(item)), item))
draw

[-2,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 '0-AddTarget-R',
 '0-AddTarget-R',
 '0-Disarm-R',
 '0-Muddle-R',
 '0-Pierce2-R',
 '0-Pierce2-R',
 '0-Push1-R',
 '0-Push1-R',
 '0-Push1-R',
 '0-Push1-R',
 '0-Push1-R',
 '0-Push1-R',
 '0-Shield',
 '0-Stun-R',
 '0-Stun-R',
 'crit',
 'null']

In [49]:
'0-Stun-R' in draw

True

In [113]:
'a' in 'abcde'

True

In [58]:
modifier = 'AddTarget'
# test_dict = {modifier: 1}
test_dict[modifier] += 1
print(test_dict)

{'AddTarget': 6}


In [85]:
added_dmg

2

In [212]:
'a--b'.split('-')

['a', '', 'b']